In [65]:
import numpy as np
import networkx as nx
from sklearn.cluster import SpectralClustering
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.utils import to_dense_adj
from torch_geometric.utils import to_networkx

In [ ]:
dataset = GNNBenchmarkDataset(root='tmp/gnn_benchmark', name='CSL')

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]
print(data)

Dataset: CSL(150):
Number of graphs: 150
Number of features: 0
Number of classes: 10


Extracting tmp/gnn_benchmark/CSL/raw/CSL.zip
Processing...
Done!


### Very basic generation of a graphon

This is much simpler than the method described in the paper, only here as a sorta placeholder/demo.

In [ ]:
G = to_networkx(data)

In [70]:
# adjaency_matrix = to_dense_adj(data.edge_index).numpy().squeeze()
adjaency_matrix = nx.to_numpy_array(G)
adjaency_matrix

array([[0., 1., 1., ..., 0., 1., 1.],
       [1., 0., 1., ..., 0., 0., 1.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [1., 0., 0., ..., 1., 0., 1.],
       [1., 1., 0., ..., 1., 1., 0.]])

In [71]:
k = 10
clustering = SpectralClustering(n_clusters=k, affinity='precomputed').fit(adjaency_matrix)
labels = clustering.labels_

In [73]:
graphon = np.zeros((k, k))
for i in range(k):
    for j in range(k):
        nodes_i = np.where(labels == i)[0]
        nodes_j = np.where(labels == j)[0]
        # print(nodes_i, nodes_j)
        # raise ValueError('TODO: implement this')
        subgraph = G.subgraph(nodes_i.tolist() + nodes_j.tolist())
        graphon[i, j] = nx.density(subgraph)